In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training
)
from datasets import Dataset
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import torch

In [2]:
from huggingface_hub import login
# Login to Hugging Face Hub
login()

In [3]:
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [4]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Toggle LoRA vs QLoRA
use_qlora = True  # set False to use standard LoRA (full precision)

if use_qlora:
    # QLoRA: 4-bit quantization via Transformers' BitsAndBytesConfig
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        quantization_config=quant_config,
        trust_remote_code=True,
    )
    model = prepare_model_for_kbit_training(model)
else:
    # Standard LoRA: full-precision adapter
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True,
    )


In [6]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # adjust based on model internals
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\peft\tuners\tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [8]:
embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(
    collection_name="database",
    embedding_function=embed_model,
    persist_directory="database"
)
texts = vectorstore.get().get("documents", [])  # list[str]
print(f"Number of documents: {len(texts)}")
hf_dataset = Dataset.from_dict({"text": texts})

Number of documents: 1731


In [11]:
def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

# Tokenize & batch
tokenized_ds = hf_dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"]
)

Map:   0%|          | 0/1731 [00:00<?, ? examples/s]

In [12]:
training_args = TrainingArguments(
    output_dir="./phase5_lora_llama3.2-1b",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    warmup_steps=100,
    num_train_epochs=3,                   
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_steps=200,
    optim="paged_adamw_8bit",
    remove_unused_columns=False,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [13]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_ds,
    args=training_args,
    data_collator=data_collator,
)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: krishshandilya18 (krishshandilya18-pes-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
20,2.802400
40,2.703600
60,2.584900
80,2.496700
100,2.445300
120,2.360700
140,2.361100
160,2.338400


TrainOutput(global_step=162, training_loss=2.510403824441227, metrics={'train_runtime': 1574.5316, 'train_samples_per_second': 3.298, 'train_steps_per_second': 0.103, 'total_flos': 1.533761789362176e+16, 'train_loss': 2.510403824441227, 'epoch': 2.9607390300230945})

In [14]:
model.save_pretrained("./phase5_lora_llama3.2-1b")
tokenizer.save_pretrained("./phase5_lora_llama3.2-1b")

('./phase5_lora_llama3.2-1b\\tokenizer_config.json',
 './phase5_lora_llama3.2-1b\\special_tokens_map.json',
 './phase5_lora_llama3.2-1b\\tokenizer.json')